# Homework 2 - Mike Atwell
####Rockhurst University
####BIA6304
####2018 Fall Term 1

##Preparation

###Package imports

First we'll import all of the packages we'll need.

In [5]:
import pandas as pd   
from bs4 import BeautifulSoup
import requests
import time
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import math
import csv
import reprlib
import string

pd.set_option('display.max_colwidth', 150) #important for getting all the text

###Definitions

Now we'll set definitions that can be used later.

In [6]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

##Task 1 

Task 1 is to "read in or create a data frame with at least one column of text to be analyzed.  This could be the text you used previously or new text. Based on the context of your dataset and the question you want to answer, identify at what processing you think is necessary (stop words, stemming, custom replacement, etc.) Compare the feature space before and after your processing."  

We will use the same data source as in Homework 1, the Department of Defense news page here: https://dod.defense.gov/News/.  The webpage has 10 headlines displayed at any one time, so if we want more than 10 to work with, we'll have to extract from multiple pages; the code below does so using a loop.  In this case, I looped the first 100 pages; over 800 are available from the website.

First let's create the dataframe using the code from before.

In [7]:
base_url = 'https://dod.defense.gov/News/Archive/?Page=' #this url is the same for the start of each page
Headline_agg = [] 
Location_agg = []
Story_Date_agg = []

for i in range(1,101):  #The range determines which page numbers are scraped
    url = f'{base_url}{i}' #appends the number of this iteration of the loop to the base url
    page = requests.get(url) 
    soup = BeautifulSoup(page.text,"html5lib")
    Headline = soup.find_all('p',{'class':'title'})  #gets first desired element: headline
    Location = soup.find_all('span',{'class':'dateline'}) #gets second desired element: location
    Story_Date = soup.find_all('p',{'class':'date'}) #gets third desired element: story date
    Headline_agg.append(Headline) #takes the headline info and adds to previous iterations of the loop
    Location_agg.append(Location) #takes the location info and adds to previous iterations of the loop
    Story_Date_agg.append(Story_Date) #takes the story date info and adds to previous iterations of the loop

We've extracted the data we want from the webpage.  Now let's format to make it useable.

In [8]:
#Formats headlines

Headline_str = str(Headline_agg) #converts to string
Headline_str = Headline_str[2:-2] #removes uneeded brackets at the beginning and end
Headline_str = remove_html_tags(Headline_str) #removes unneeded html tags
Headline_str = (re.sub(',', '',Headline_str)) #next two lines remove unneeded characters
Headline_str = (re.sub('  +', '',Headline_str)) 
Headline_str = Headline_str.replace(' ','_') #substitutes spaces within headlines for _ so they won't be split up on the next line of code
Headline_str = Headline_str.split() #splits the string so that each headline is a different data element
#print(Headline_str)    #Troubleshooting tool

In [9]:
#Formats locations

Location_str = str(Location_agg) #converts to string
Location_str = Location_str.replace('—','') #removes unneeded dash in location name
Location_str = remove_html_tags(Location_str) #removes unneeded html tags
Location_str = Location_str[2:-2] #removes uneeded brackets at the beginning and end
Location_str = (re.sub(',', '',Location_str)) #next two lines remove unneeded characters
Location_str = (re.sub('  +', '',Location_str))
Location_str = Location_str.replace(' ','_') #substitutes spaces within headlines for _ so they won't be split up on the next line of code
Location_str = Location_str.split() #splits the string so that each headline is a different data element
#print(Location_str)    #Troubleshooting tool

In [10]:
#Formats story date

Story_Date_str = str(Story_Date_agg) #converts to string
Story_Date_str = remove_html_tags(Story_Date_str) #removes unneeded html tags
Story_Date_str = Story_Date_str[2:-2] #removes uneeded brackets at the beginning and end
Story_Date_str = (re.sub(',', '',Story_Date_str)) #next two lines remove unneeded characters
Story_Date_str = (re.sub('  +', '',Story_Date_str))
Story_Date_str = Story_Date_str.replace(' ','_') #substitutes spaces within headlines for _ so they won't be split up on the next line of code
Story_Date_str = Story_Date_str.replace('.','') #removes unneeded character
Story_Date_str = Story_Date_str.split() #splits the string so that each headline is a different data element
#print(Story_Date_str)    #Troubleshooting tool

Now the data is ready to be put in a dataframe.  The formatting is mostly complete, but there's a few more steps to be done. 

In [11]:
headline_data = {'Headline':Headline_str,'Location':Location_str,'Story Date':Story_Date_str}  #assigns column names
headlinedf = pd.DataFrame(headline_data) #creates dataframe
headlinedf = headlinedf.replace({'_': ' '}, regex=True) #substitutes the _ that we added earlier for a space now that the data is split
headlinedf['Access Date'] = (time.strftime("%b %d %Y")) #adds a column date for when the data was scraped
headlinedf = headlinedf.drop(headlinedf[headlinedf.Headline == '] ['].index)  #since multiple pages were scraped, this drops unecessary rows of brackets between them
headlinedf = headlinedf.reset_index(drop=True) #renumbers rows since some were dropped in the last line
pd.set_option('display.max_rows', 10) #since the dataframe is potentially very long, this restricts output to a managable length
headlinedf #shows the head and tail of the dataframe

,Headline,Location,Story Date,Access Date
0,Northcom Commander: Response Framework Working Well in Florence Aftermath,WASHINGTON,Sept 18 2018,Sep 18 2018
1,Coalition Effort Aims at Stability in Iraq Syria,WASHINGTON,Sept 18 2018,Sep 18 2018
2,New Biodefense Strategy Combats Man-Made Natural Threats,WASHINGTON,Sept 18 2018,Sep 18 2018
3,Defense Intel Chief Puts Great Power Competition in Context,WASHINGTON,Sept 18 2018,Sep 18 2018
4,Coalition Soldiers Train Iraqi Mortar Troops,IRAQ,Sept 18 2018,Sep 18 2018
...,...,...,...,...
995,Dunford Visits With Australian Chief of Defense Moves on to Thailand,RAAF DARWIN Australia,Feb 6 2018,Sep 18 2018
996,Coalition Announces Shift in Focus as Iraq Campaign Progresses,SOUTHWEST ASIA,Feb 5 2018,Sep 18 2018
997,Field Artillery Armor Troops Train Together in Germany,GRAFENWOEHR TRAINING AREA Germany,Feb 5 2018,Sep 18 2018
998,Air Force Logisticians Fuel Air Missions,AL UDEID AIR BASE Qatar,Feb 5 2018,Sep 18 2018


We now have a dataframe with the desired info.  In homework 1, we applied a filter of those words relating to different countries.  We didn't just use country names as different forms of the name may be used in a headline, e.g. 'Argentina' or 'Argentine'.  I used this page: https://www.cia.gov/library/publications/the-world-factbook/fields/2110.html which has a useful list of different variations on a country's name or its people in different grammatical contexts, and put those words in a csv file.

That said, though all country-related words made it through the filter, all variant spellings on a country or its people were counted differently, so (for instance) 'Iraq', 'Iraqi', and 'Iraqis' were not summed together, creating a distorted view of just how many times a country was mentioned in Defense Department Headlines.  In the above dataframe output, in row 995 we see 'Ukrainian', exemplfying this very issue.

Additionally, since the bag of words used singular words, this complicated getting a word count for a country with two or more words like 'United Kingdom'; the words 'United' and 'Kingdom' were counted separately, thus the counts of the individual words were lumped in with other contributers such as 'United States' or 'Kingdom of Saudi Arabia'.

To correct these problems, we'll use a conversion table that converts multi word mentions of a country or continent to a single word (using an underscore), and also converts all variants on a country's mention to the country name itself (e.g. 'French' to 'France').  Thus, it should provide a much more accurate count of the country's appearance in Defense Department headlines than was obtained in homework 1.  The table is in csv format, so first we have to convert it to a usable dictionary.

Since we're eventually going to filter out all but country-related words, I saw no need to use stop words or to put words in lowercase.  Additionally, I didn't think stemming would work as it wouldn't pare all country related words the same to make tallying useful or accurate; compare 'Afghan', 'Afghanistan', and 'Afghans', these wouldn't all pare down to the same word in a stemming algorithm.   

Before applying the conversion, let's get the feature space size of the raw headlines.

In [12]:
textStr = headlinedf['Headline'] #creates a generic name for the headline text; useful for reusing the code for other projects 

cv1 = CountVectorizer(binary=True, lowercase=False) #this will state only if a word appears in the headline or not; we don't want to double count a country's mention in a headline

cv1_chat = cv1.fit_transform(textStr) #transforms text

print(type(cv1_chat))
print(cv1_chat.shape) #gets feature space size

<class 'scipy.sparse.csr.csr_matrix'>
(1000, 2371)


We see that over 1,000 headlines, 2,374 different words are used.  Now let's import the country conversion table and format it to be a usable dictionary.  I used a space after each word to avoid part of a word being transformed.

In [13]:
with open('CountryWordListConversion1.csv', 'r') as f:  #opens the csv file with the conversion table
  reader = csv.reader(f)
  country_list_converter = list(reader) #creates the csv input as a list   
    
country_list_converter = country_list_converter[1:] #deletes header row from the list   
country_list_converter_dict = dict(country_list_converter) #converts list to dictionary

print("Dictionary Length:", len(country_list_converter_dict)) #prints the length of the dictionary
print(reprlib.repr(country_list_converter_dict))  #prints first few dictonary items to verify success

Dictionary Length: 485
{'Afghan ': 'Afghanistan ', 'Afghani ': 'Afghanistan ', 'Afghanis ': 'Afghanistan ', 'Afghans ': 'Afghanistan ', ...}


The dictionary is ready; now let's convert the headlines using it.

In [14]:
pd.set_option('display.max_rows', 10) #sets output to desired length

#converts country words per the dictionary
for origword, newword in country_list_converter_dict.items():   
    headlinedf.Headline = headlinedf.Headline.str.replace(origword, newword)
    
headlinedf

,Headline,Location,Story Date,Access Date
0,Northcom Commander: Response Framework Working Well in Florence Aftermath,WASHINGTON,Sept 18 2018,Sep 18 2018
1,Coalition Effort Aims at Stability in Iraq Syria,WASHINGTON,Sept 18 2018,Sep 18 2018
2,New Biodefense Strategy Combats Man-Made Natural Threats,WASHINGTON,Sept 18 2018,Sep 18 2018
3,Defense Intel Chief Puts Great Power Competition in Context,WASHINGTON,Sept 18 2018,Sep 18 2018
4,Coalition Soldiers Train Iraq Mortar Troops,IRAQ,Sept 18 2018,Sep 18 2018
...,...,...,...,...
995,Dunford Visits With Australia Chief of Defense Moves on to Thailand,RAAF DARWIN Australia,Feb 6 2018,Sep 18 2018
996,Coalition Announces Shift in Focus as Iraq Campaign Progresses,SOUTHWEST ASIA,Feb 5 2018,Sep 18 2018
997,Field Artillery Armor Troops Train Together in Germany,GRAFENWOEHR TRAINING AREA Germany,Feb 5 2018,Sep 18 2018
998,Air Force Logisticians Fuel Air Missions,AL UDEID AIR BASE Qatar,Feb 5 2018,Sep 18 2018


The headlines have been transformed per the dictionary.  Compare row 995, it's now 'Ukraine' vice 'Ukrainian'.  Now that the transformation has been done, let's get the feature space size.

In [15]:
textStr = headlinedf['Headline'] #creates a generic name for the headline text; useful for reusing the code for other projects 

cv2 = CountVectorizer(binary=True, lowercase=False) #this will state only if a word appears in the headline or not; we don't want to double count a country's mention in a headline

cv2_chat = cv2.fit_transform(textStr) #transforms text

print(type(cv2_chat))
print(cv2_chat.shape) #gets feature space size

<class 'scipy.sparse.csr.csr_matrix'>
(1000, 2346)


We see that the country transformation table shrank the feature space from 2,372 to 2,348 words.  That may not seem like much but now we'll filter out all but country-related words.  First let's import the country list, which matches format with the post-conversion dictionary.

In [16]:
pd.set_option('display.max_rows', 8) #sets output to desired length
country_list = pd.read_csv("FinalCountryWordList.csv", header=None) #imports csv
country_list

,0
0,Afghanistan
1,Africa
2,Albania
3,Algeria
...,...
233,Wallis_and_Futuna
234,Yemen
235,Zambia
236,Zimbabwe


Now we'll compare this list with the words in the headlines to get a count of country mentions.

In [17]:
names = cv2.get_feature_names()   #create list of feature names, after conversion

count = np.sum(cv2_chat.toarray(), axis = 0) # convert list to array to add up feature counts 
count2 = count.tolist()  # convert numpy array to list

data = {'country': names, 'count': count2} #columns for dataframe
count_df = pd.DataFrame(data=data) # create a dataframe from the list

pd.set_option('display.max_rows', 25) #sets output to desired length

set1 = set(count_df['country']) #all words in the headlines
set2 = set(country_list[0]) #all countries and country-related words 
set3 = set2 - set1  #all countries not mentioned in headlines
set4 = set2 - set3  #all countries mentioned in the headlines

count_df = count_df.loc[count_df['country'].isin(set4)] #reduces dataframe rows only to those countries in headlines 
count_df = count_df.set_index('country') #removes unneeded row numbers
count_df = count_df.sort_values(by = ['count'], ascending=False) #sorts values in descending order

print(count_df.shape) #prints number of country-related words after the filter is applied
count_df.head(25) #prints top 25 results

(90, 1)


,count
country,
Syria,61
Iraq,57
Afghanistan,34
Japan,10
Australia,9
Russia,9
India,8
Georgia,8
Vietnam,8


The country filter shrank the feature space to just 90 words.

We see 'South_Korea' and 'North_Korea' are mentioned as entities in their own right; 'Korea' appears as well when mentioned in a headline without 'North' or 'South' as context, whereas in Homework 1 all of these were lumped together.  

Additionally, related words such as 'Iraq' and 'Iraqi' are now counted together, producing a more accruate picture of its frequency being mentioned, and similarly other countries such as Japan and Russia appear much higher on the list than they were before.  

Thus, it appears the word list worked as intended.

##Task 2

Task 2 is to "create a sentiment dictionary from one of the sources in class or find/create your own (potential bonus points for appropriate creativity). Using your dictionary, create sentiment labels for the text entries in your corpus."

We going to try the AFINN sentiment dictonary for our purposes, which grades (on a scale) a positive or negative score depending on the word.  First we'll define it from the file of words and scores: 

In [18]:
afinn = {}
for line in open("AFINN-111.txt"):
    tt = line.split('\t')
    afinn.update({tt[0]:int(tt[1])})

#print(type(afinn), len(afinn))

#for key, value in sorted(afinn.items())[0:10]:
#    print(key + " => " + str(value))
#print("~~~~~~~~~~~~")
#for key, value in sorted(afinn.items())[2467:]:
#    print(key + " => " + str(value))

For our analysis we will get a total score for the sentiment of the headlines a country is mentioned in; thus it makes sense to use a sentiment score (e.g. using a positive number vice the word 'positive') so we can do math operations on it later.  This next code defines how to caluculate the sentiment count:

In [19]:
def afinn_sent(inputstring):
    
    sentcount =0
    for word in inputstring.split():  
        if word.rstrip('?:!.,;') in afinn:
            sentcount = sentcount + afinn[word.rstrip('?:!.,;')]
            
    
    #if (sentcount < 0):
    #    sentiment = 'Negative'
    #elif (sentcount >0):
    #    sentiment = 'Positive'
    #else:
    #    sentiment = 'Neutral'
    
    #return sentiment
    return sentcount

Now we'll create a dataframe to capture the sentiment count for each headline:

In [20]:
headlinedf_sent = pd.DataFrame(data=headlinedf['Headline']) #creates new dataframe to record sentiment

#next code populates the sentiment count for each headline.  Used lower case in the code as the 
#sentiment dictionary is in lower case
headlinedf_sent['Sentiment Count'] = headlinedf_sent.Headline.str.lower().apply(lambda x: afinn_sent(x))

pd.set_option('display.max_rows', 30) #sets output to desired length
headlinedf_sent

,Headline,Sentiment Count
0,Northcom Commander: Response Framework Working Well in Florence Aftermath,0
1,Coalition Effort Aims at Stability in Iraq Syria,0
2,New Biodefense Strategy Combats Man-Made Natural Threats,-2
3,Defense Intel Chief Puts Great Power Competition in Context,3
4,Coalition Soldiers Train Iraq Mortar Troops,0
5,Kentucky Air Guard Deploys Search and Recovery Team Following Hurricane Florence,0
6,Face of Defense: Marines Enjoy Duty as Military Police Working Dog Handlers,2
7,Mattis Praises Macedonia as ‘Stabilizing Force’,3
8,Coalition Strikes Continue Against ISIS Targets in Iraq Syria,-1
9,Medics Test Mental Physical Endurance,0


Now we'll create the bag of words to get a word count by headline.

In [21]:
textStr = headlinedf_sent['Headline'] #creates a generic name for the headline text; useful for reusing the code for other projects 

cv3 = CountVectorizer(binary=True, lowercase=False) #this will state only if a word appears in a sentence or not; therefore it doesn't matter if the word is used more than once in a headline

cv3_chat = cv3.fit_transform(textStr) #transforms text

word_count = pd.DataFrame(cv3_chat.toarray(),columns = cv3.get_feature_names())

pd.set_option('display.max_rows', 10) #sets output to desired length
word_count

,100,101st,10th,11,116,12000,14th,16,167th,17,...,it,its,of,on,positioning,than,the,to,with,years
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,1,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now let's screen out all but the country names from the count, as this is the info we're interested in.

In [22]:
cols = [col for col in word_count.columns if col in set4]
word_count_country_only = word_count[cols]
word_count_country_only

,Afghanistan,Africa,Albania,Argentina,Asia,Australia,Bahrain,Botswana,Brazil,Bulgaria,...,Thailand,Togo,Turkey,Ukraine,United_Kingdom,United_States,Uzbekistan,Venezuela,Vietnam,Yemen
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We have a word count of country related names only.  Let's multiply the count by the Sentiment Count for each headline obtained earlier to 'score' each country mention across all headlines.

In [23]:
sent_count = headlinedf_sent['Sentiment Count'].tolist()
sentiment_scoring_by_country = word_count_country_only.multiply(sent_count, axis=0)
pd.set_option('display.max_rows', 10) #sets output to desired length
sentiment_scoring_by_country

,Afghanistan,Africa,Albania,Argentina,Asia,Australia,Bahrain,Botswana,Brazil,Bulgaria,...,Thailand,Togo,Turkey,Ukraine,United_Kingdom,United_States,Uzbekistan,Venezuela,Vietnam,Yemen
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now let's sum the columns and see the overall country sentiment!

In [24]:
pd.set_option('display.max_rows', 20) #sets output to desired length
total_score = sentiment_scoring_by_country.sum() #add the sum of the sentiment scores by country
total_score.sort_values(ascending=False)

Afghanistan         25
Colombia            11
India                9
Europe               9
United_States        8
Philippines          6
Africa               5
Mexico               5
Venezuela            5
Norway               4
                    ..
China               -3
Marshall_Islands    -3
Niger               -3
North_Korea         -3
Georgia             -4
Korea               -4
Somalia             -4
Russia              -6
Iraq               -24
Syria              -40
Length: 90, dtype: int64

We see that 'Afghanistan' has a large positive sentiment, while Syria and Iraq have large negative sentiments.  This is surprising; since the U.S. is aiding both Afghanistan and Iraq (as opposed to the Syrian government) it seems unfair that Iraq's sentiment should be so negative while Afghanistan's is so positive.  Let's take a closer look at Iraq.

##Task 3

Task 3 is "consider one of the entries in your corpus that had a surprising label.  How would you change your analysis to get the “right” label? Show specific results."

We want to find out why Iraq is getting such a large negative sentiment count and see what we can do to adjust it.  First let's narrow down the headlines to only those that contained Iraq and a non-zero sentiment score to see the context.

In [25]:
iraq_only = headlinedf_sent[headlinedf_sent.Headline.str.contains("Iraq")]
iraq_only = iraq_only[iraq_only['Sentiment Count'] != 0] 
iraq_only

,Headline,Sentiment Count
8,Coalition Strikes Continue Against ISIS Targets in Iraq Syria,-1
39,Coalition Strikes Target ISIS Terrorists in Syria Iraq,-1
68,Coalition Strikes Continue Against ISIS Targets in Iraq Syria,-1
95,Coalition Strikes Continue Against ISIS Targets in Iraq Syria,-1
122,Military Aircraft Crash in Iraq Highlights Dangers Troops Face Each Day,-2
154,Coalition Strikes Continue Against ISIS Targets in Iraq Syria,-1
186,Coalition Strikes Target ISIS Terrorists in Syria Iraq,-1
216,Coalition Strikes Continue Against ISIS Targets in Iraq Syria,-1
243,Coalition Strikes Continue Against ISIS Targets in Syria Iraq,-1
261,Iraq Kurdish Troops Clear ISIS Safe Haven,2


We see that Iraq (along with Syria) is mentioned in numerous headlines mentioning strikes against ISIS.  Looking at the AFINN sentiment dictionary, we see that 'strikes' has this score:    

In [26]:
print(afinn["strikes"])

-1


With a score of -1, this explains much of the negative sentiment score for Iraq and Syria.  It seems unfair to 'punish' Iraq's sentiment rating for strikes against ISIS, and in any event the Defense Department typically uses 'strikes' in the context of U.S. or allied forces conducting attacks (not enemy attacks), so let's try adjusting the sentiment dictionary to better capture the intent of this.  Let's remove 'strikes' from the sentiment dictionary and see the impact. 

In [27]:
afinn_mod = dict(afinn)
del afinn_mod["strikes"]

The dictionary is now modified without the word 'strikes'.  Let's run the sentiment analysis again and see the result.  First we redefine how to get sentiment count with the new dictionary: 

In [28]:
def afinn_sent_mod(inputstring):
    
    sentcount =0
    for word in inputstring.split():  
        if word.rstrip('?:!.,;') in afinn_mod:
            sentcount = sentcount + afinn_mod[word.rstrip('?:!.,;')]
 
    return sentcount

Now we'll create a dataframe to capture the modified sentiment count for each headline:

In [29]:
headlinedf_sent_mod = pd.DataFrame(data=headlinedf['Headline']) #creates new dataframe to record sentiment

#next code populates the sentiment count for each headline.  Used lower case in the code as the 
#sentiment dictionary is in lower case
headlinedf_sent_mod['Sentiment Count'] = headlinedf_sent_mod.Headline.str.lower().apply(lambda x: afinn_sent_mod(x))

pd.set_option('display.max_rows', 10) #sets output to desired length
headlinedf_sent_mod

,Headline,Sentiment Count
0,Northcom Commander: Response Framework Working Well in Florence Aftermath,0
1,Coalition Effort Aims at Stability in Iraq Syria,0
2,New Biodefense Strategy Combats Man-Made Natural Threats,-2
3,Defense Intel Chief Puts Great Power Competition in Context,3
4,Coalition Soldiers Train Iraq Mortar Troops,0
...,...,...
995,Dunford Visits With Australia Chief of Defense Moves on to Thailand,0
996,Coalition Announces Shift in Focus as Iraq Campaign Progresses,0
997,Field Artillery Armor Troops Train Together in Germany,0
998,Air Force Logisticians Fuel Air Missions,0


We see one 'strikes' headline with a sentiment score of zero.  Now we'll create the bag of words to get a word count by headline.

In [30]:
textStr = headlinedf_sent_mod['Headline'] #creates a generic name for the headline text; useful for reusing the code for other projects 

cv4 = CountVectorizer(binary=True, lowercase=False) #this will state only if a word appears in a sentence or not; therefore it doesn't matter if the word is used more than once in a headline

cv4_chat = cv4.fit_transform(textStr) #transforms text

word_count = pd.DataFrame(cv4_chat.toarray(),columns = cv4.get_feature_names())

pd.set_option('display.max_rows', 10) #sets output to desired length
word_count

,100,101st,10th,11,116,12000,14th,16,167th,17,...,it,its,of,on,positioning,than,the,to,with,years
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,1,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now let's screen out all but the country names from the count, as this is the info we're interested in.

In [31]:
cols = [col for col in word_count.columns if col in set4]
word_count_country_only = word_count[cols]
word_count_country_only

,Afghanistan,Africa,Albania,Argentina,Asia,Australia,Bahrain,Botswana,Brazil,Bulgaria,...,Thailand,Togo,Turkey,Ukraine,United_Kingdom,United_States,Uzbekistan,Venezuela,Vietnam,Yemen
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We have a word count of country related names only.  Let's multiply the count by the modified Sentiment Count for each headline obtained earlier to 'score' each country mention across all headlines.

In [32]:
sent_count = headlinedf_sent_mod['Sentiment Count'].tolist()
sentiment_scoring_by_country = word_count_country_only.multiply(sent_count, axis=0)
pd.set_option('display.max_rows', 10) #sets output to desired length
sentiment_scoring_by_country

,Afghanistan,Africa,Albania,Argentina,Asia,Australia,Bahrain,Botswana,Brazil,Bulgaria,...,Thailand,Togo,Turkey,Ukraine,United_Kingdom,United_States,Uzbekistan,Venezuela,Vietnam,Yemen
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now we'll sum the columns and see the overall country sentiment using the modified dictionary.

In [33]:
pd.set_option('display.max_rows', 20) #sets output to desired length
total_score = sentiment_scoring_by_country.sum() #add the sum of the sentiment scores by country
total_score.sort_values(ascending=False)

Afghanistan         25
Colombia            11
India                9
Europe               9
United_States        8
Philippines          6
Africa               5
Venezuela            5
Mexico               5
United_Kingdom       4
                    ..
Djibouti            -2
China               -3
North_Korea         -3
Niger               -3
Marshall_Islands    -3
Somalia             -4
Georgia             -4
Korea               -4
Russia              -6
Syria              -10
Length: 90, dtype: int64

We see Iraq isn't in the bottom ten results anymore, and Syria's score has improved considerably, though it still has the most negative sentiment.  Let's see what Iraq's new score is:

In [34]:
print(total_score.Iraq)

3


Iraq now has a small positive sentiment score.